### Import Libraries

In [1]:
from Server_Model import *
from Server_Encryption import *
from flask import Flask, json, request, render_template, session, redirect
from flask import send_file
from urllib.parse import urlparse
import requests
from cryptography.fernet import Fernet
from random import randrange
from flask import session, url_for

### Flask APIs

In [2]:
def check_available_clients():
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    
    available_clients = []
    for c in client_list:
        r = requests.get(url = "http://"+c+'/active_test')
    
        #print(r, r.status_code, r.reason, r.text)
        if r.status_code == 200:
            #print("yeah")
            available_clients.append(c)
            
    return available_clients

In [ ]:
api = Flask(__name__)


@api.route('/', methods=['GET'])
def get_landing_page():
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    if len(client_list) > 0:
        return render_template('ServerLanding.html', list_clients=list(client_list))
    return "No clients"

@api.route('/register', methods=['GET'])
def register_client():
    
    port = request.args.get('port')
    url = request.args.get('url')
    
    try:
        with open("client.list", "r") as f:
            client_list = f.read()   
    except:
        client_list = ""
        
    client_list = client_list.split("\n")
    client_list.append(url + ":" + port)
    client_list = list(set(client_list))
    client_list = '\n'.join(str(x) for x in client_list)
    
    with open("client.list", "w") as f:
        f.write(client_list.strip())
    
    serverack = {'server_ack': '1'}
    return str(serverack)


@api.route('/send_weights_to_clients', methods=['GET'])
def send_weights_to_clients():
    
    model_aggregation()
    clients = check_available_clients()
    order_file_path, file_name = encrypt_file('agg_model.npy')
    
    with open('filekey.key', 'rb') as filekey:
        key = filekey.read()

    fernet = Fernet(key)
    with open(order_file_path, 'rb') as r:
        data = r.read()
        
    data = fernet.encrypt(data = data)
    
    with open(order_file_path, 'wb+') as w:
        w.write(data)
        
    file = open(file_name, 'rb')
    order_file = open(order_file_path, 'rb')
    
#     with open('filekey.key', 'rb') as filekey:
        
    
    data = {'fname':'agg_model.npy', 'order_file':'order_file'}
    files = {
        'json': ('json_data', json.dumps(data), 'application/json'),
        'model': ('agg_model.npy', file, 'application/octet-stream'),
        'order_file': ('order_file', order_file, 'application/octet-stream')
    }
    
    for c in clients:
        try:
            req = requests.post(url="http://"+c+'/downloadmodel', files=files)
        except:
            pass
    
    file.close()
    order_file.close()
    os.remove(order_file_path)
    os.remove(file_name)
    
    return "Aggregated model sent !"

@api.route('/client_model', methods=['POST'])
def getmodel():
    if request.method == 'POST':
        file = request.files['model'].read()
        order_file = request.files['order_file'].read()
        data = request.files['json'].read()
        print(data)
        
        with open('filekey.key', 'rb') as filekey:
            key = filekey.read()
        
        fernet = Fernet(key)
        order_file = fernet.decrypt(order_file)
        print(order_file)
        data = ast.literal_eval(data.decode("utf-8"))

        order_file_name = data['order_file']
        cli = data['url']+'_' + str(data['port'])
        fname = cli +"_" + data['fname']

        print(fname)
        wfile = open("client_models_"+fname, 'wb+')
        wfile.write(file)
        wfile.close()
        
        wfile = open("client_models_"+order_file_name, 'wb+')
        wfile.write(order_file)
        wfile.close()
        
        decrypt_file('client_models_'+fname, 'client_models_'+order_file_name)  
        os.remove('client_models_'+order_file_name)
        
        return "Model received!"
    else:
        return "No file received!"

if __name__ == '__main__':
    api.secret_key = randrange(100000)
    api.config['SESSION_TYPE'] = 'filesystem'
    api.config.update(
        SESSION_COOKIE_NAME = 'session_server',
        SESSION_COOKIE_PATH = '/server/'
    )
    api.run(port=10011)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:10011/ (Press CTRL+C to quit)


10000 test samples
['client_models_127.0.0.1_5000_agg_model.npy']
client_models_127.0.0.1_5000_agg_model.npy
Test loss: 0.0271899476647377
Test accuracy: 0.9918000102043152
Model written to storage!


C:\Users\pk15425\Anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
127.0.0.1 - - [14/Jul/2022 23:36:51] "GET /send_weights_to_clients HTTP/1.1" 200 -
127.0.0.1 - - [14/Jul/2022 23:38:38] "POST /client_model HTTP/1.1" 200 -


b'{"fname": "agg_model.npy", "url": "127.0.0.1", "port": 5000, "order_file": "order_file"}'
b'{"0": 1, "1": 0, "2": 3, "3": 7, "4": 2, "5": 5, "6": 6, "7": 4}'
127.0.0.1_5000_agg_model.npy
10000 test samples
['client_models_127.0.0.1_5000_agg_model.npy']
client_models_127.0.0.1_5000_agg_model.npy


127.0.0.1 - - [14/Jul/2022 23:38:49] "GET /send_weights_to_clients HTTP/1.1" 200 -


Test loss: 0.03229633718729019
Test accuracy: 0.9911999702453613
Model written to storage!
